# Adding dummy var features regarding ellipses

>1. Load original data
>2. Create a function for 0 dummy var
>3. Create a function for 1 dummy var
>4. Create a function for 6 dummy var
>5. Create a function for 8 dummy var
>6. Create a function for 9 dummy var
>7. Load intermediate data and append onto that

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.svm as skl_svm
import sklearn.cross_validation as skl_cv
import seaborn as sns
sns.set_context(context = 'notebook', font_scale=1.5)
import os
import sys
import cv2

base_path = '/home/lundi/Python/MNIST/'
sys.path.append(base_path + '/libraries/')

import MNIST_data_processor as mdp

MNIST_data_processor = mdp.MNIST_data_processor()

from matplotlib.patches import Ellipse

## 1. Load original data

In [17]:
X, y = MNIST_data_processor.load_subset_data()

## 2. Create a function for 0 dummy var

In [18]:
def convert_to_image(data):
    img = np.zeros((28, 28,3))
    img[:,:,0] = data.reshape(28,28)
    img[:,:,1] = data.reshape(28,28)
    img[:,:,2] = data.reshape(28,28)
    
    img = img.astype('uint8')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return img

In [62]:
def get_distance_between_ellipses(ellipse_1, ellipse_2):
    return np.sqrt(((ellipse_1[0][0] - ellipse_2[0][0]) ** 2) + ((ellipse_1[0][1] - ellipse_2[0][1]) ** 2))

In [88]:
def get_0_dummy_var(contour_data):

    valid_contours = []
    #Get the valid contours
    for cnt in contour_data:
        if len(cnt) >= 5:
            valid_contours.append(cnt)
            
    if len(valid_contours) == 2:
        ellipse = []
        ellipse_1 = cv2.fitEllipse(valid_contours[0])
        ellipse_2 = cv2.fitEllipse(valid_contours[1])
                
        distance = get_distance_between_ellipses(ellipse_1, ellipse_2)
        
        return [(ellipse_1[1][0] * ellipse_1[1][1] * np.pi), (ellipse_2[1][0] * ellipse_2[1][1] * np.pi)]
        #print 'xy', ellipse[0]
        #print 'area:', ellipse[1][0] * ellipse[1][1] * np.pi
        #print 'W:', ellipse[1][0]
        #print 'H:', ellipse[1][1]
    else:
        return -1
            


In [89]:
def get_dummy_vars(digit_data):
    
    image = convert_to_image(digit_data)
    
    ret,thresh = cv2.threshold(image,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    return get_0_dummy_var(contours)

In [90]:
result_data = pd.concat([X.apply(get_dummy_vars, axis=1), y], axis=1)

In [91]:
result_data

,0,label
0,-1,1
1,"[503.572450221, 1014.48710255]",0
2,-1,1
3,-1,4
4,"[528.310765236, 1201.18004215]",0
5,"[452.298437405, 654.933784677]",0
6,-1,7
7,-1,3
8,-1,5
9,-1,3


In [83]:
result_data.replace(-1, np.nan).dropna().groupby(['label'])[0].describe()

Series([], Name: 0, dtype: float64)

## 3. Create a function for 1 dummy var

## 4. Create a function for 6 dummy var

## 5. Create a function for 8 dummy var

## 6. Create a function for 9 dummy var

## 7. Load intermediate data and append onto that